In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import pandas_profiling as pp
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
!pip install pycaret

In [ ]:
# Importing our dataset
df=pd.read_csv('../input/heart-disease-uci/heart.csv')

In [ ]:
df.columns

In [ ]:
# Toh doston,Chaliye shuru karte hain 
# Let us start our EDA
df.head()

In [ ]:
df

In [ ]:
df.info()

In [ ]:
# Checking for missing values
df.isnull().sum()


Ok so we have no missing values. 

In [ ]:
# Let us seperate our target variable from the features so that we can build a model. We will be using a Supervised learning 
# approach in this notebook by making using of different classifiers, cross-validating,among other things.


x=df.iloc[:,:-1].values
y=df.target


In [ ]:
# let us look at x and y

x

In [ ]:
y

In [ ]:
# let us look at the correlation of different features with the target variable

matrix=df.corr()
sns.heatmap(matrix)

In [ ]:
print(matrix['target'])

As you can see features are not highly correlated with the target varibale. The max correlation is 0.433 for the feature namely 'cp'

Let us now look at individual d=features in depth by using the pandas_profiling library. See below

In [ ]:
profile = pp.ProfileReport(df, title='Heart Disease Dataset Report')

In [ ]:
profile

> Go through the above report generated using pandas-profiling to get better insight about individual features.



In [ ]:
# let us look at the skew of different features 
print(df.skew())

In [ ]:
# Let us plot all the features except the one with binary values or repeating values ('I am doing this to viualise the variance in 
#  variance exhibited by other fetures')
# Creating a new dataframe and dropping all the columns that are not required in the visual
df1=df.drop([ 'sex', 'cp',   'fbs', 'restecg', 
       'exang',  'slope', 'ca', 'thal'],axis=1)
sns.pairplot(df1,hue='target')

We can use a log tranformation on the dataset to reduce the skew and try to transform the datset as closely as possible to a normal distribution. However let us first try by using some classification models and then let us see what we shall do next.

In [ ]:
df.dtypes

In [ ]:
from pycaret.classification import * 
exp = setup(data = df, target = 'target', session_id=1,
                  normalize = True,
                categorical_features = [ 'sex', 'cp',  'fbs', 'restecg', 
       'exang',  'slope' , 'thal', ],
                numeric_features=['age','trestbps','chol','thalach','oldpeak','ca'],
                categorical_imputation='mode',
                numeric_imputation='mean',
            remove_outliers=True,
            outliers_threshold=0.1,
            normalize_method='robust',
            feature_selection=True,
            feature_selection_threshold=0.9,
            remove_multicollinearity=True,
            train_size=0.8
            
                )

So our PyCaret setup is now successful. I have set the train size at 80 % and teest size at 20%. You can play around with these values to find the ideal ratio. Let us now comapre our models.


In [ ]:
compare_models()

As you can see the results obtained were not that great.The maximum accuracy that we obtained was around 82 % with KNN classifier and a 10 fold cross validation. Let us see what happens if we dont remove the outliers.

In [ ]:
from pycaret.classification import * 
exp = setup(data = df, target = 'target', session_id=1,
                  normalize = True,
                categorical_features = [ 'sex', 'cp',  'fbs', 'restecg', 
       'exang',  'slope' , 'thal', ],
                numeric_features=['age','trestbps','chol','thalach','oldpeak','ca'],
                categorical_imputation='mode',
                numeric_imputation='mean',
            normalize_method='robust',
            feature_selection=True,
            feature_selection_threshold=0.9,
            remove_multicollinearity=True,
            train_size=0.8
            
                )

In [ ]:
compare_models()

An instant increase of 2% in the accuracy . Let us now use only a select few features based on their correlation and see if this affects our model in anyway.